### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [114]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [115]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma-old', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [116]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  2.23200011253
Compiling results...
Done.

Total time (s):  4.78900003433 



### Or make all fits

In [124]:
p_dict = {
    'order' : {
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [125]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor', 'ma-old']:
    for order_fit in ['nlo', 'nnlo', 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  3.90799999237
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.59699988365 

xpt nnlo
Using default prior.
Making fits...
100% complete Time (s):  9.82100009918
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.3760001659 

xpt nnnlo
Using default prior.
Making fits...
100% complete Time (s):  16.1560001373
Compiling results...
Done.
Saving...
Done.

Total time (s):  18.8450000286 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  3.96199989319
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.4359998703 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  8.96299982071
Compiling results...
Done.
Saving...
Done.

Total time (s):  11.5520000458 

xpt-taylor nnnlo
Using default prior.
Making fits...
100% complete Time (s):  14.5509998798
Compiling results...
Done.
Saving...
Done.

Total time (s):  17.1460001469 

ma nlo
Using d

In [92]:
#bootstrapper.plot_parameter_histogram('L_5')

In [86]:
#print bootstrapper

In [87]:
#print bootstrapper.fits[0]

In [214]:
#bootstrapper.fits[0].show_plots()